In [62]:
import os 
from pathlib import Path

import pandas as pd
import numpy as np


In [63]:
from sbscorer.sblegos.TransactionAnalyser import TransactionAnalyser as txa

In [64]:
FOLDER_NAME = 'gr18'
CHAIN = 'optimism'
extract_all = True

# Set path to data folder
current_dir = Path(os.getcwd())
PATH_TO_EXPORT = os.path.join(current_dir.parent.parent, 'tx_data', FOLDER_NAME)
PATH_TO_GR18 = os.path.join(current_dir.parent.parent, FOLDER_NAME)
voter_tx_file_name = 'gg18_votes.csv'
passport_file_name = 'gg18_voter_passports.csv'

tx_file_name = 'gr_18_optimism_all_tx.csv'

In [65]:
votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))
passports = pd.read_csv(os.path.join(PATH_TO_GR18, passport_file_name))

C:\Users\DELL Admin\AppData\Local\Temp\ipykernel_20732\1560736169.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))


In [66]:
round_names = votes['round_name'].unique()
round_names

array(['Climate Round', 'Web3 Open Source Software',
       'Web3 Community and Education ',
       'Global Chinese Community for Public Goods - GR18', 'Web3 Social',
       'Zuzalu Continuous Innovation', 'Meta Pool LatAm GG18',
       'ReFi DAO Local Node ', 'Token Engineering',
       'Ethereum Infrastructure', 'Arbitrum Domain Round'], dtype=object)

In [67]:
# Choose round name to compute features
round_name = round_names[0]

In [68]:
# Load tx data
df_tx = pd.read_csv(os.path.join(PATH_TO_EXPORT, tx_file_name))
df_tx.head()

,tx_hash,block_timestamp,from_address,to_address,gas_limit,gas_used,tx_fee,eth_value,__row_index
0,0x863c5b543d29b329891fdec341845a73c2525e6dc212...,2023-08-24T20:22:03.000Z,0xf4af0314a06ea7376da5bdecc33f6f9800f1ae28,0x7f5c764cbc14f9669b88837ca1490cca17c31607,53067,53067,0.000029,0.000000,0
1,0xfdffb6eb77e72ca9f54297ebcc7644ea55db521f64c7...,2023-08-24T22:05:41.000Z,0x87dbffd0ed90b2252856faf753437a7bc9fcfa7b,0xb009b5707e6e0dc9830a1262de37f055972e6480,219895,139765,0.000058,0.000777,1
2,0x7b416f625fd29c2da17a7ad35b1c6dc7e0cf5b548249...,2023-08-24T16:10:17.000Z,0x532c1fc2940ec38d5c5e7fc8b8aa4aaddcaa17d3,0x46ce3b08bdef4461fb444d3746fd97f0af11dccf,731592,467423,0.000093,0.000000,2
3,0xbed4ce09f5f5c4d7aac7e310dfce11d94bf86f7da6d5...,2023-08-24T16:09:33.000Z,0x0596c217a79ea00e5dafc43778138ea61fd67ee2,0x2e42f214467f647fe687fd9a2bf3baddfa737465,119625,119625,0.000087,0.000000,3
4,0xb60502458c8de31606b84e0aed102ac83629726d8734...,2023-08-24T22:08:15.000Z,0xc1991c8bda29507991eeb230ff2063c2ea74a34c,0xa062ae8a9c5e11aaa026fc2670b0d65ccc8b2858,246899,229592,0.000070,0.000000,4


In [69]:
CONTRACT_CREATION_AD_NAME = '0x_contract_creation'

In [70]:
df_tx.loc[df_tx['to_address'].isna(), 'to_address'] = CONTRACT_CREATION_AD_NAME

In [71]:
df_tx.isna().sum().sum()

tx_hash            0
block_timestamp    0
from_address       0
to_address         0
gas_limit          0
gas_used           0
tx_fee             0
eth_value          0
__row_index        0
dtype: int64

In [72]:
df_tx.drop(columns=['__row_index'], inplace=True)

In [73]:
df_tx.shape

(6776216, 8)

In [74]:
votes_round = votes.loc[votes['round_name'] == round_name, :].copy()

In [75]:
round_voters = votes_round['voter'].unique()
print(f'Number of voters in round {round_name}: {len(round_voters)}')

Number of voters in round Web3 Open Source Software: 39694


In [76]:
df_tx_EOA_FROM = df_tx.copy()
df_tx_EOA_FROM['EOA'] = df_tx_EOA_FROM['from_address']

In [77]:
df_tx_EOA_TO = df_tx.copy()
df_tx_EOA_TO['EOA'] = df_tx_EOA_TO['to_address']

In [78]:
round_voters

array(['0xC5f17c199Bd1215Ff0730106FEb890B60356cF8a',
       '0xe9CDc2fb0f61C3AF9dB6f2A01CDfD10083de955C',
       '0x00D8d69A862C94B7Ec6F8DF1eF30599CF31D39f2', ...,
       '0xC259FFd187F28E9Fd651099e51bF5c9Ebe8b3C99',
       '0xfEA3aEa4A085FBebE600d2B49B12e5C6C1D39e76',
       '0x350830cb7B67598564b6D5e64742AF06A953b15d'], dtype=object)

In [79]:
df_tx_eoa = pd.concat([df_tx_EOA_FROM, df_tx_EOA_TO])

In [80]:
df_tx_eoa.shape

(13552432, 9)

In [81]:
df_tx_eoa = df_tx_eoa.loc[df_tx_eoa['EOA'] != CONTRACT_CREATION_AD_NAME, :]

In [82]:
df_tx_eoa.shape

(13551371, 9)

In [90]:
lowercase_converter = np.vectorize(str.lower)
round_voters = lowercase_converter(round_voters)

In [91]:
tx_analyser = txa(df_tx_eoa, round_voters)

In [98]:
tx_analyser.df_transactions.shape

(6352901, 9)

In [100]:
len(tx_analyser.array_address)

39559

In [101]:
len(round_voters)

39694

In [102]:
np.setdiff1d(round_voters, tx_analyser.array_address)

array(['0x0000006d14ce3cf81449c3ba1f26108df0a4de8b',
       '0x005988f679bc17119f9fa7b592fc13982b8b181d',
       '0x063ae79a21d7794f69d2afae38b73bb9a957efd3',
       '0x0736878acec9ab4ad1f84cc2686644b68a8a6c56',
       '0x08f14aba09e0d146cfd862f677d5d53413cfcf01',
       '0x0b39264ccd02540820cec61bb3ae55b3b787190e',
       '0x0b8d19b7249904f38fba58dc7bb46d7f849e19b6',
       '0x0bf6d1f70240f467cebfa57093f9ac35f4cec409',
       '0x0fd955d29d88a2b7258f8b364cdfe36c2996438a',
       '0x100beacaa540a85f8abf0c328d6b6e01e429c48c',
       '0x102da0207ba3e1b18fcc826fde188a133e0d27d4',
       '0x1514139d56b7ba3497994484b8d801ca17ea8909',
       '0x172783c29f5c3e56ebc33b798a69e16a974b22fc',
       '0x1c6a6ebac96864e60502ce6f38e3de47590e03e1',
       '0x1e2c6e02d10b305d92bf856a336fa920cfdc20f7',
       '0x2017ed424993783060b861832a012541a6e489bd',
       '0x2184f80e9dfdd832f7cfaf3cb01817a8bf6d8056',
       '0x22ef71c7e4cbddfddb61e793217246c128533505',
       '0x2565cb8914d312934a9dd18f034417636bed

In [92]:
df_features = tx_analyser.get_df_features()

In [93]:
df_features.shape

(39559, 19)

In [96]:
df_features

,EOA,count_tx,less_10_tx,count_same_seed,count_same_seed_naive,same_seed,same_seed_naive,seed_suspicious,count_interact_other_ctbt,first_in_tx_from,first_in_tx_gas_limit,first_in_tx_gas_used,first_in_tx_eth_value,first_in_tx_timestamp,to_address,first_out_tx_gas_limit,first_out_tx_gas_used,first_out_tx_eth_value,first_out_tx_timestamp
0,0x00000001985c6a08963eef53821c78aa84318b42,111,False,8109,0,True,False,True,0,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,200000.0,21000.0,0.100000,2023-01-08T11:43:30.000Z,0x7f5c764cbc14f9669b88837ca1490cca17c31607,64083.0,53403.0,0.000000,2023-01-06T07:54:58.000Z
1,0x00000001b01122945d67cf7c972f1a2063ca4008,219,False,0,0,False,False,False,0,NaN,NaN,NaN,NaN,NaN,0xdef1abe32c034e558cdd535791643c58a13acc10,365387.0,279544.0,0.000739,2022-09-06T08:22:24.000Z
2,0x0000000452bbfc5541279949e5e9400821a0c92d,98,False,0,0,False,False,False,0,NaN,NaN,NaN,NaN,NaN,0x7f5c764cbc14f9669b88837ca1490cca17c31607,58743.0,53403.0,0.000000,2023-01-07T03:56:25.000Z
3,0x000000064730640b7d670408d74280924883064f,278,False,8109,0,True,False,True,0,0xacd03d601e5bb1b275bb94076ff46ed9d753435a,200000.0,21000.0,5.877201,2022-12-01T15:24:40.000Z,0xb8df6cc3050cc02f967db1ee48330ba23276a492,3851430.0,180986.0,0.005000,2021-11-15T20:50:24.000Z
4,0x00000009d94baf057697d988d2e3eec2fce81162,220,False,0,0,False,False,False,0,0x646e9a6a66726d54fa1137df4448fda922c50f0d,21000.0,21000.0,0.000486,2023-03-30T06:44:18.000Z,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,184830.0,129391.0,0.001000,2022-09-06T06:11:33.000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39554,0xfff9b8f876d3e30fe305806738176c4fe04594f0,18,False,174,0,True,False,True,0,0x9ef21be1c270aa1c3c3d750f458442397fbffcb6,21000.0,21000.0,0.039000,2023-08-05T13:13:23.000Z,0xb49c4e680174e331cb0a7ff3ab58afc9738d5f8b,434585.0,369984.0,0.047337,2023-04-15T22:57:08.000Z
39555,0xfff9e8387f6ea2073d1c38b9571fe4aa55fd3ad8,274,False,3772,2374,True,True,False,0,0x2d2cc0eb095e43204e0c087e07dbf95909650939,150000.0,21000.0,0.102000,2022-06-23T12:06:51.000Z,0x86ca30bef97fb651b8d866d45503684b90cb3312,277262.0,213825.0,0.051000,2022-06-23T12:12:59.000Z
39556,0xfffb43b6b6048adc32ded4fcedc429b17e7ea72d,308,False,0,0,False,False,False,0,0xdf362413e0125a93c57c31472f5cc02bebb99306,21000.0,21000.0,0.010000,2022-11-28T14:10:50.000Z,0x8c6f28f2f1a3c87f0f938b96d27520d9751ec8d9,84130.0,69706.0,0.000000,2022-11-29T12:44:21.000Z
39557,0xfffd05ae9db4d5f4e23e2f37395db2f87359bf5d,396,False,0,0,False,False,False,0,NaN,NaN,NaN,NaN,NaN,0x9270b79d24d8eb1bba8cf7774ad6c23f2e8eac0e,458832.0,452179.0,0.000000,2022-11-20T05:36:40.000Z


In [104]:
df_features.columns

Index(['EOA', 'count_tx', 'less_10_tx', 'count_same_seed',
       'count_same_seed_naive', 'same_seed', 'same_seed_naive',
       'seed_suspicious', 'count_interact_other_ctbt', 'first_in_tx_from',
       'first_in_tx_gas_limit', 'first_in_tx_gas_used',
       'first_in_tx_eth_value', 'first_in_tx_timestamp', 'to_address',
       'first_out_tx_gas_limit', 'first_out_tx_gas_used',
       'first_out_tx_eth_value', 'first_out_tx_timestamp'],
      dtype='object')

In [106]:
df_features['count_interact_other_ctbt'].value_counts()

count_interact_other_ctbt
0     31914
1      4270
2      1650
3       726
4       370
5       195
6       136
7       113
8        69
10       32
9        31
11       20
12       14
14        6
15        3
13        3
16        2
17        1
18        1
59        1
50        1
21        1
Name: count, dtype: int64

In [107]:
df_features[df_features['count_interact_other_ctbt'] >= 15]

,EOA,count_tx,less_10_tx,count_same_seed,count_same_seed_naive,same_seed,same_seed_naive,seed_suspicious,count_interact_other_ctbt,first_in_tx_from,first_in_tx_gas_limit,first_in_tx_gas_used,first_in_tx_eth_value,first_in_tx_timestamp,to_address,first_out_tx_gas_limit,first_out_tx_gas_used,first_out_tx_eth_value,first_out_tx_timestamp
3864,0x18df3723c654571fc91acee13ee55178bb94dc57,182,False,0,1,False,True,True,17,0xb6e32176c77faba8dd12cf319a412aad989b0107,21000.0,21000.0,0.001000,2022-03-11T16:08:06.000Z,0x68f180fcce6836688e9084f035309e29bf0a2095,55950.0,46625.0,0.00,2022-03-11T16:08:41.000Z
4083,0x1a676d7fe4b97f1b10b1407600602f9fbf516ae8,348,False,5770,2342,True,True,False,15,0x80c67432656d59144ceff962e8faf8926599bcf8,100000.0,21000.0,0.034817,2022-12-22T09:40:28.000Z,0x1e04c33cd5a015e1ced0e3ecd8bdc42902512124,129085.0,122555.0,0.00,2022-12-22T09:42:59.000Z
7451,0x3047bf8e87f0786702b8e8265c3b04cedba920e3,190,False,0,18,False,True,True,18,NaN,NaN,NaN,NaN,NaN,0x0bfb21f64e414ff616ac54853e52679eedb22dd2,93725.0,78725.0,0.00,2021-12-14T15:42:59.000Z
8743,0x38cbd13d538ba6207d57f479efbe8261f775773e,146,False,2,3,True,True,False,16,0x9e32c96cd497653ea98adcf608c7ac961ad18522,21000.0,21000.0,0.009300,2022-03-10T20:42:53.000Z,0x68f180fcce6836688e9084f035309e29bf0a2095,46625.0,46625.0,0.00,2022-03-10T20:45:53.000Z
8986,0x3a5bd1e37b099ae3386d13947b6a90d97675e5e3,222,False,0,0,False,False,False,59,0xd755a8d0c72b379f445ab17fe60af9a76eaad221,21000.0,21000.0,0.001000,2023-07-27T15:20:45.000Z,0xfedfaf1a10335448b7fa0268f56d2b44dbd357de,88838.0,88838.0,0.00,2022-06-01T03:48:37.000Z
26926,0xacd54246eda0c5638a247c4717a8782c21497eb6,1624,False,0,5,False,True,True,16,0x078a0b6d78d6fa119655fe6a1b7c02fc9dc7ff73,53000.0,21000.0,0.016278,2022-11-19T21:57:09.000Z,0xedd118e091eee0e7fa9eeb9b4db717518f56cb15,830818.0,328043.0,0.06,2022-11-10T11:30:40.000Z
28121,0xb4bb4a094338bc566e4bb0355c9de525f7c1f1b4,1061,False,1265,835,True,True,False,50,0xf89d7b9c864f589bbf53a82105107622b35eaa40,21000.0,21000.0,0.006998,2022-06-08T17:14:58.000Z,0xfedfaf1a10335448b7fa0268f56d2b44dbd357de,88954.0,88954.0,0.00,2022-06-08T17:17:07.000Z
28609,0xb80bd21033cc4dcfdc9dfe5ec9e46cabd4128d8d,468,False,0,0,False,False,False,21,0x07fd387ae14f8ec8fcc89ff1efa60154f40692f4,21000.0,21000.0,0.009000,2021-12-30T04:16:12.000Z,0xe592427a0aece92de3edee1f18e0157c05861564,258649.0,179542.0,0.00,2021-12-30T06:18:09.000Z
29289,0xbcc306cff05cd3a24286591bfc98efb443144a90,767,False,1,0,True,False,True,15,0x43a234c2c9f698d7dc56cfb32b7dfac31f7fe0de,21000.0,21000.0,0.002000,2022-10-17T18:09:52.000Z,0x81b30ff521d1feb67ede32db726d95714eb00637,169808.0,169808.0,0.00,2022-07-28T19:18:07.000Z
30612,0xc55b8da10e52e9f7bf464f4c85a59f686dd7d772,611,False,0,0,False,False,False,15,NaN,NaN,NaN,NaN,NaN,0x81b30ff521d1feb67ede32db726d95714eb00637,169808.0,169808.0,0.00,2022-07-28T11:48:52.000Z


In [103]:
df_features['less_10_tx'].sum()

7982

In [95]:
df_features.to_csv(os.path.join(PATH_TO_EXPORT, f'features_{round_name}.csv'), index=False)